In [1]:
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [31]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score,train_test_split,GridSearchCV

from sklearn.tree import DecisionTreeClassifier

# from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, \
ConfusionMatrixDisplay, precision_score, recall_score, f1_score, \
classification_report, roc_curve, precision_recall_curve, average_precision_score

In [3]:
df = pd.read_csv('../stroke_data_cleaned.csv')
df

,Unnamed: 0,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_gr,health_status,age_gr
0,0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1,Extremely Obese,hd_only,Old
1,1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1,Overweight,No disease,Old
2,2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1,Obese,hd_only,Old
3,3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1,Obese,No disease,Mature Adult
4,4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1,Normal,ht only,Old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,5105,Female,80.0,1,0,Yes,Private,Urban,83.75,28.893237,never smoked,0,Overweight,ht only,Old
5106,5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.000000,never smoked,0,Extremely Obese,No disease,Old
5107,5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.600000,never smoked,0,Obese,No disease,Young Adult
5108,5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.600000,formerly smoked,0,Overweight,No disease,Mature Adult


In [4]:
df = df[df.columns[1:12]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                5110 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [12]:
df.isnull().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [24]:
df2 = df.copy()

In [25]:
gender_labels = {'Male':0,'Female':1}
ever_married_labels = {'Yes':0,'No':1}
work_type_labels = {'Private' : 0, 'Self-employed' : 1, 'Govt_job' : 2, 'children' : 3, 'Never_worked' : 4}
residence_type_labels = {'Urban' : 0, 'Rural' : 1}
smoking_status_labels = {'formerly smoked' : 0, 'never smoked' : 1, 'smokes' : 2, 'Unknown' : 3}

df2['gender'] = df2['gender'].map(gender_labels)
df2['ever_married'] = df2['ever_married'].map(ever_married_labels)
df2['work_type'] = df2['work_type'].map(work_type_labels)
df2['Residence_type'] = df2['Residence_type'].map(residence_type_labels)
df2['smoking_status'] = df2['smoking_status'].map(smoking_status_labels)

df2.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,0,0,0,228.69,36.600000,0,1
1,1,61.0,0,0,0,1,1,202.21,28.893237,1,1
2,0,80.0,0,1,0,0,1,105.92,32.500000,1,1
3,1,49.0,0,0,0,0,0,171.23,34.400000,2,1
4,1,79.0,1,0,0,1,1,174.12,24.000000,1,1


In [26]:
X = df2.drop(columns='stroke').values
y = df2['stroke'].values

In [27]:
X

array([[  0.        ,  67.        ,   0.        , ..., 228.69      ,
         36.6       ,   0.        ],
       [  1.        ,  61.        ,   0.        , ..., 202.21      ,
         28.89323691,   1.        ],
       [  0.        ,  80.        ,   0.        , ..., 105.92      ,
         32.5       ,   1.        ],
       ...,
       [  1.        ,  35.        ,   0.        , ...,  82.99      ,
         30.6       ,   1.        ],
       [  0.        ,  51.        ,   0.        , ..., 166.29      ,
         25.6       ,   0.        ],
       [  1.        ,  44.        ,   0.        , ...,  85.28      ,
         26.2       ,   3.        ]])

In [28]:
y

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [29]:
X = MinMaxScaler().fit_transform(X)
X

array([[0.        , 0.81689453, 0.        , ..., 0.80126489, 0.30126002,
        0.        ],
       [1.        , 0.74365234, 0.        , ..., 0.67902317, 0.21298095,
        0.33333333],
       [0.        , 0.97558594, 0.        , ..., 0.23451205, 0.25429553,
        0.33333333],
       ...,
       [1.        , 0.42626953, 0.        , ..., 0.12865848, 0.2325315 ,
        0.33333333],
       [0.        , 0.62158203, 0.        , ..., 0.51320284, 0.17525773,
        0.        ],
       [1.        , 0.53613281, 0.        , ..., 0.13922999, 0.18213058,
        1.        ]])

In [33]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=5)

In [35]:
print('shape of X_train:',X_train.shape)
print('shape of X_test:',X_test.shape)
print('shape of y_train:',y_train.shape)
print('shape of y_test:',y_test.shape)

shape of X_train: (3577, 10)
shape of X_test: (1533, 10)
shape of y_train: (3577,)
shape of y_test: (1533,)


In [36]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(x_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

Before OverSampling, counts of label '1': 174
Before OverSampling, counts of label '0': 3403 

After OverSampling, the shape of train_X: (6806, 10)
After OverSampling, the shape of train_y: (6806,) 

After OverSampling, counts of label '1': 3403
After OverSampling, counts of label '0': 3403


In [37]:
classifier = DecisionTreeClassifier(random_state=8)
classifier.fit(x_train_res,y_train_res)

y_pred = classifier.predict(X_test)

In [38]:
cm = confusion_matrix(y_test,y_pred)
accuracies = cross_val_score(estimator = classifier,
                            X = x_train_res,
                            y = y_train_res,
                            cv=10)
roc = roc_auc_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred)

In [40]:
print('classification report')
print(classification_report(y_test,y_pred))
print('K-fold validation mean accuracy')
print(accuracies.mean()*100)
print('roc score:')
print(roc)

classification report
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1458
           1       0.06      0.11      0.07        75

    accuracy                           0.87      1533
   macro avg       0.50      0.51      0.50      1533
weighted avg       0.91      0.87      0.89      1533

K-fold validation mean accuracy
91.39032132676859
roc score:
0.5073799725651578


In [44]:
param_grid = {
    'max_depth':[3,5,10,20],
    'min_samples_leaf':[5,10,20,50,100],
    'criterion':['entropy','gini']
    }

In [46]:
grid_search = GridSearchCV(
    estimator = classifier,
    param_grid = param_grid,
    cv = 10,
    scoring = 'accuracy'
)
grid_search.fit(x_train_res,y_train_res)
best_accuracy = grid_search.best_score_
best_param = grid_search.best_params_
print('Best Accuracy : {:.2f}%'.format(best_accuracy*100))
print('Best Parameters : ',best_param)

Best Accuracy : 89.80%
Best Parameters :  {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 5}


In [48]:
classifier = DecisionTreeClassifier(
    criterion='entropy',
    max_depth = 20,
    min_samples_leaf = 5,
    random_state = 35
)

classifier.fit(x_train_res,y_train_res)
y_pred = classifier.predict(X_test)

In [49]:
print('classification report')
print(classification_report(y_test,y_pred))

classification report
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      1458
           1       0.13      0.24      0.17        75

    accuracy                           0.88      1533
   macro avg       0.54      0.58      0.55      1533
weighted avg       0.92      0.88      0.90      1533

